# Hyperparameter Tuning

In this notebook, you'll learn how to tune the hyperparameters for a machine learning algorithm. The specific scenario in this exercise is to find the optimal hyperparameter values for training a model that predicts the species of a penguin based on observed features.

> **Citation**: The penguins dataset used in the this exercise is a subset of data collected and made available by [Dr. Kristen
Gorman](https://www.uaf.edu/cfos/people/faculty/detail/kristen-gorman.php)
and the [Palmer Station, Antarctica LTER](https://pal.lternet.edu/), a
member of the [Long Term Ecological Research
Network](https://lternet.edu/).

## Ingest data

Run the following cell to ingest the data file you will use in this exercise. The data file will be saved in the DBFS storage for your Azure Databricks cluster.

In [ ]:
%sh
rm -r /dbfs/data
mkdir /dbfs/data
wget -O /dbfs/data/penguins.csv https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks-ml/master/data/penguins.csv

## Prepare the data
  
Now let's prepare the data for machine learning. Run the following cell to:

1. Remove any incomplete rows
2. Apply appropriate data types
3. View a random sample of the data
4. Split the data into two datasets: one for training, and another for testing.

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

data = spark.read.format("csv").option("header", "true").load("/data/penguins.csv")
data = data.dropna().select(col("Island").astype("string"),
                          col("CulmenLength").astype("float"),
                          col("CulmenDepth").astype("float"),
                          col("FlipperLength").astype("float"),
                          col("BodyMass").astype("float"),
                          col("Species").astype("int")
                          )
display(data.sample(0.2))

splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]
print ("Training Rows:", train.count(), " Testing Rows:", test.count())

The data consists of measurements of the following details of penguins that have been observed in Antartica:

- **Island**: The island in Antartica where the penguin was observed.
- **CulmenLength**: The length in mm of the penguin's culmen (bill).
- **CulmenDepth**: The depth in mm of the penguin's culmen.
- **FlipperLength**: The length in mm of the penguin's flipper.
- **BodyMass**: The body mass of the penguin in grams.
- **Species**: An integer value that represents the species of the penguin:
  - **0**: *Adelie*
  - **1**: *Gentoo*
  - **2**: *Chinstrap*
  
Our goal in this project is to use the observed charateristics of a pengun (its *features*) in order to predict its species (which in machine learning terminology, we call the *label*).

## Optimize hyperparameter values for training a model

You train a machine learning model by fitting the features to an algorithm that calculates the most probable label. Algorithms take the training data as a parameter and attempt to calculate a mathematical relationship between the features and labels. In addition to the data, most algorithms use one or more *hyperparameters* to influence the way the relationship is calculated; and determining the optimal hyperparameter values is an important part of the iterative model training process.

To help you determine optimal hyperparameter values, Azure Databricks includes support for **Hyperopt** - a library that enables you to try multiple hyperparameter values and find the best combination for your data.

### Create a training function

The first step in using Hyperopt is to create a function that:

- Trains a model using one or more hyperparameter values that are passed to the function as parameters.
- Calculates a performance metric that can be used to measure *loss* (how far the model is from perfect prediction performance)
- Returns the loss value so can be optimized (minimized) iteratively by trying different hyperparameter values

Run the following cell to create a function that does this for the penguins data.

In [ ]:
from hyperopt import STATUS_OK
import mlflow
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, MinMaxScaler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

def objective(params):
    # Train a model using the provided hyperparameter value
    catFeature = "Island"
    numFeatures = ["CulmenLength", "CulmenDepth", "FlipperLength", "BodyMass"]
    catIndexer = StringIndexer(inputCol=catFeature, outputCol=catFeature + "Idx")
    numVector = VectorAssembler(inputCols=numFeatures, outputCol="numericFeatures")
    numScaler = MinMaxScaler(inputCol = numVector.getOutputCol(), outputCol="normalizedFeatures")
    featureVector = VectorAssembler(inputCols=["IslandIdx", "normalizedFeatures"], outputCol="Features")
    mlAlgo = DecisionTreeClassifier(labelCol="Species", featuresCol="Features",
                                    maxDepth=params['MaxDepth'], maxBins=params['MaxBins'])
    pipeline = Pipeline(stages=[catIndexer, numVector, numScaler, featureVector, mlAlgo])
    model = pipeline.fit(train)
    
    # Evaluate the model to get the target metric
    prediction = model.transform(test)
    eval = MulticlassClassificationEvaluator(labelCol="Species", predictionCol="prediction", metricName="accuracy")
    accuracy = eval.evaluate(prediction)
    
    # Hyperopt tries to minimize the objective function, so you must return the negative accuracy.
    return {'loss': -accuracy, 'status': STATUS_OK}


### Iteratively run the function to optimize hyperparameters

Now that you have a function defined, you need to call it iteratively with different combinations of hyperparameter values.

To do this, you need to:

- Define a search space that specifies the range of values to be used for one or more hyperparameters (see [Defining a Search Space](http://hyperopt.github.io/hyperopt/getting-started/search_spaces/) in the Hyperopt documentation for more details).
- Specify the Hyperopt algorithm you want to use (see [Algorithms](http://hyperopt.github.io/hyperopt/#algorithms) in the Hyperopt documentation for more details).
- Use the **hyperopt.fmin** function to call your training function repeatedly and try to minimize the loss.

Run the following cell to optimize hyperparameters for your penguin classification training function (this may take some time!)

In [ ]:
from hyperopt import fmin, tpe, hp

# Define a search space for two hyperparameters (maxDepth and maxBins)
search_space = {
    'MaxDepth': hp.randint('MaxDepth', 10),
    'MaxBins': hp.choice('MaxBins', [10, 20, 30])
}

# Specify an algorithm for the hyperparameter optimization process
algo=tpe.suggest

# Call the training function iteratively to find the optimal hyperparameter values
argmin = fmin(
  fn=objective,
  space=search_space,
  algo=algo,
  max_evals=6)

print("Best param values: ", argmin)


The code will iteratively run the training function 6 times (based on the **max_evals** setting).
Each run is recorded by MLflow, and you can use the the **&#9656;** toggle to expand the **MLflow run** output under the code cell and select the **experiment** hyperlink to view them. Each run is assigned a random name, and you can view each of them in the MLflow run viewer to see details of parameters and metrics that were recorded.

When all of the runs have finished, the code displays details of the best hyperparameter values that were found (the combination that resulted in the least loss). In this case, the **MaxBins** parameter is defined as a choice from a list of three possible values (10, 20, and 30) - the best value indicates the zero-based item in the list (so 0=10, 1=20, and 2=30). The **MaxDepth** parameter is defined as a random integer between 0 and 10, and the integer value that gave the best result is displayed. For more information about specifying hyperparameter value scopes for search spaces, see [Parameter Expressions](http://hyperopt.github.io/hyperopt/getting-started/search_spaces/#parameter-expressions) in the Hyperopt documentation.

## Use the Trials class to log run details

In addition to using MLflow experiment runs to log details of each iteration, you can also use the **hyperopt.Trials** class to record and view details of each run.

In [ ]:
from hyperopt import Trials

# Create a Trials object to track each run
trial_runs = Trials()

argmin = fmin(
  fn=objective,
  space=search_space,
  algo=algo,
  max_evals=3,
  trials=trial_runs)

print("Best param values: ", argmin)

# Get details from each trial run
print ("trials:")
for trial in trial_runs.trials:
    print ("\n", trial)

In this notebook, you've been introduced to **Hyperopt** as a way to optimize hyperparameter values for machine learning models.

For more information see [Hyperparameter tuning](https://learn.microsoft.com/azure/databricks/machine-learning/automl-hyperparam-tuning/) in the Azure Databricks documentation.